# 신경망 모델 구성하기

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## 학습을 위한 장치 얻기

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


## 클래스 정의하기

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [14]:
X = torch.rand(1, 28, 28, device = device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([3], device='cuda:0')


## 모델 계층

In [15]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


## nn.Flatten

In [16]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


## nn.Linear

In [19]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


## nn.ReLU

In [20]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.2289,  0.3832,  0.1689, -0.7058,  0.6603,  0.3002,  0.3466, -0.0116,
          0.0968,  0.3081, -0.1973,  0.0754,  0.6930,  0.2795,  0.1954,  0.3647,
         -0.1209,  0.2220, -0.0198,  0.2744],
        [-0.3199,  0.3079,  0.0947, -0.8440,  0.3970,  0.3265,  0.4842, -0.0391,
          0.3711,  0.3320, -0.0123,  0.2756,  0.9561,  0.1876,  0.1148,  0.2172,
         -0.3306,  0.2244, -0.1208,  0.0176],
        [-0.1270,  0.4255, -0.1491, -0.5371,  0.3570, -0.0103,  0.7380,  0.2601,
          0.1152,  0.6357, -0.2804, -0.1226,  0.6802,  0.1062,  0.2895,  0.1325,
         -0.5971,  0.1069, -0.1320,  0.0832]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.3832, 0.1689, 0.0000, 0.6603, 0.3002, 0.3466, 0.0000, 0.0968,
         0.3081, 0.0000, 0.0754, 0.6930, 0.2795, 0.1954, 0.3647, 0.0000, 0.2220,
         0.0000, 0.2744],
        [0.0000, 0.3079, 0.0947, 0.0000, 0.3970, 0.3265, 0.4842, 0.0000, 0.3711,
         0.3320, 0.0000, 0.2756, 0.9561, 0.1876, 0.11

## nn.Seqiential

In [22]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

## nn.Softmax

In [24]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## 모델 매개변수

In [25]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0284, -0.0136, -0.0250,  ..., -0.0246,  0.0292,  0.0083],
        [-0.0267,  0.0355,  0.0181,  ..., -0.0071, -0.0232, -0.0338]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0243,  0.0105], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0069, -0.0238,  0.0233,  ..., -0.0225,  0.0087, -0.0146],
        [ 0.0321, -0.0172,  0.0238,  ..., -0.0029,  0.0266,  0.0355]],
       device='cuda:0', grad_fn=<Slice